In [216]:
import numpy as  np
import pandas as pd
import keras as keras

# to plot within notebook
import matplotlib.pyplot as plt
%matplotlib inline
df = pd.read_csv('deng_train.csv')

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split

In [217]:
# transforming dataframe and scaling data
# any blanks were replaced by median in excel
traindf = pd.get_dummies(df)

In [218]:
# correlation matrix
corr = traindf.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)
# 'RdBu_r' and 'BrBG' is another colour mapping

In [219]:
# create results df
results = traindf.loc[:,['total_cases']]

In [220]:
# remove y values from feature df
newtraindf = traindf.drop(columns=['total_cases'])
newtraindf.head(1)

,year,weekofyear,datevalue,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precip_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,monthofyear_June,monthofyear_March,monthofyear_May,monthofyear_November,monthofyear_October,monthofyear_September,season_Autumn,season_Spring,season_Summer,season_Winter
0,1990,18,30,0.1226,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,...,0,0,0,0,0,0,0,1,0,0


In [221]:
x_train, x_test, y_train, y_test = train_test_split(newtraindf, results, test_size = 0.25)

In [222]:
ridge = Ridge()
ridge.fit(x_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [223]:
lasso = Lasso()
lasso.fit(x_train, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [224]:
ridge_pred = round(pd.DataFrame(ridge.predict(x_test)), 0)
lasso_pred = round(pd.DataFrame(lasso.predict(x_test)), 0)

In [225]:
ridge_err = mean_absolute_error(ridge_pred, y_test)
ridge_err

17.532967032967033

In [226]:
lasso_err = mean_absolute_error(lasso_pred, y_test)
lasso_err

18.024725274725274

In [227]:
# submission values, filling in missing data
dfsub = pd.read_csv('deng_test_x.csv')
dfsub = dfsub.fillna(dfsub.median())
dfsub.head(1)

,city,year,weekofyear,monthofyear,season,datevalue,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,2008,18,April,Spring,29,29/04/2008,-0.0189,-0.0189,0.102729,...,25.37,78.781429,78.6,15.918571,3.128571,26.528571,7.057143,33.3,21.7,75.2


In [228]:
traindf = dfsub.drop(columns=['week_start_date'])
traindf = pd.get_dummies(traindf)
traindf.head(1)

,year,weekofyear,datevalue,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,...,monthofyear_June,monthofyear_March,monthofyear_May,monthofyear_November,monthofyear_October,monthofyear_September,season_Autumn,season_Spring,season_Summer,season_Winter
0,2008,18,29,-0.0189,-0.0189,0.102729,0.0912,78.6,298.492857,298.55,...,0,0,0,0,0,0,0,1,0,0


In [236]:
ridge_pred = round(pd.DataFrame(ridge.predict(traindf)), 0)
ridge_pred[ridge_pred < 0] = 0
lasso_pred = round(pd.DataFrame(lasso.predict(traindf)), 0)
lasso_pred[lasso_pred < 0] = 0

In [239]:
lasso_sub = dfsub[dfsub.columns[0:3]]
lasso_sub['total_cases'] = lasso_pred
lasso_sub.to_csv('Lasso_sub.csv')

C:\Users\jacka\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [240]:
ridge_sub = dfsub[dfsub.columns[0:3]]
ridge_sub['total_cases'] = ridge_pred
ridge_sub.to_csv('Ridge_sub.csv')

C:\Users\jacka\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
